In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
import pickle
import openpyxl
import xlwt

# Load Model

In [2]:
with open("Pickle_RFS_Model.pkl" , 'rb') as file:  
    model= pickle.load(file)

with open("Feature_Selected_RFS" , 'rb') as file2:  
    feature = pickle.load(file2)

# Load Data

In [3]:
#load training data
df = pd.read_excel("testDatasetExample.xls", index_col = False)

# Prepare Data for Prediction

In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
X_test = df[feature]

In [6]:
X_test = pd.DataFrame(StandardScaler().fit_transform(X_test.values), columns=X_test.columns, index=X_test.index)

# Prediction

In [7]:
result = model.predict(X_test)

In [8]:
df.insert(1, 'RelapseFreeSurvival (outcome)', result)

In [9]:
df.shape

(3, 119)

# Convert to Excel

In [10]:
df.to_excel('FinalTestRFS.xls', sheet_name='RFS prediction')


C:\Users\Ga401\AppData\Local\Temp\ipykernel_53992\469449715.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel('FinalTestRFS.xls', sheet_name='RFS prediction')
